In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz raspagem no site da Câmara dos Deputados para pegar partidos atuais de deputados
#

In [1]:
from urllib.request import urlopen

In [2]:
from bs4 import BeautifulSoup

In [3]:
import urllib.request, urllib.parse, urllib.error

In [2]:
import pandas as pd

In [5]:
# Acessa base com sites dos deputados - base obtida no robofoto.ipynb

In [21]:
links = pd.read_csv("dia22_11_politicos.csv",sep=',',encoding = 'utf_8')

In [22]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 6 columns):
Nome_site         342 non-null object
Num_referencia    342 non-null float64
Link              513 non-null object
Link_foto         513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 24.1+ KB


In [23]:
# Cria uma lista com os partidos dos deputados e sinalizando se estão no cargo ou não

In [ ]:
def prepara_sopa(url):
    html = urlopen(url)
    sopa = BeautifulSoup(html, "lxml")
    return sopa

lista_deputados = []

for linha, row in links.iterrows():
    exato = 0 
    nocargo = "Sim"
    url = (row['Link'])
    #print(url)
    cpf = (row['CPF'])
    nome = (row['Nome_site'])
    sopa = prepara_sopa(url)
    
    # Seleciona o trecho onde está o partido
    partido_uf_tipo = (sopa.find("ul",{"class":"visualNoMarker"}).
         find_all("li")[2].getText())
    
    localiza = partido_uf_tipo.find(':')
    exato = localiza + 1
    isola = partido_uf_tipo[exato:]
    
    tamanho = (len(isola)-14)
    
    partido_atual = isola[:tamanho].strip()
    #print(partido_atual)
    
    if sopa.findAll(text="*** não está em exercício ***"):
        nocargo = "Não"
    else:
        nocargo = "Sim"
        
    #print(nocargo)

    lista_deputados.append([url, cpf, nome, partido_atual, nocargo])

In [25]:
# Cria dataframe a partir da lista

In [26]:
deputados_partidos = pd.DataFrame(lista_deputados, columns = ['Link', 'CPF', 'Nome', 'Partido_atual', 'Esta_no_cargo'])

In [27]:
deputados_partidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
Link             513 non-null object
CPF              513 non-null int64
Nome             342 non-null object
Partido_atual    513 non-null object
Esta_no_cargo    513 non-null object
dtypes: int64(1), object(4)
memory usage: 20.1+ KB


In [28]:
deputados_partidos.to_csv('dia23_11_partidos.csv')

In [29]:
# Verifica quais não são deputados eleitos em 2014 - ou que têm nome urna diferente

In [30]:
candidatos = pd.read_csv("candidatos_nome_urna_csv.csv",sep=';',encoding = 'latin_1')

In [31]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26245 entries, 0 to 26244
Data columns (total 7 columns):
Estado            26245 non-null object
Cargo             26245 non-null object
Nome_candidato    26245 non-null object
CPF               26245 non-null int64
Nome_urna         26245 non-null object
Partido           26245 non-null object
Situacao          26245 non-null object
dtypes: int64(1), object(6)
memory usage: 1.4+ MB


In [32]:
eleitos = candidatos[(candidatos['Situacao'] == 'ELEITO POR QP') | (candidatos['Situacao'] == 'ELEITO POR MÉDIA') | (candidatos['Situacao'] == 'ELEITO')]

In [33]:
eleitos_d = eleitos[(eleitos['Cargo'] == 'DEPUTADO FEDERAL')]

In [34]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 23 to 26219
Data columns (total 7 columns):
Estado            513 non-null object
Cargo             513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null int64
Nome_urna         513 non-null object
Partido           513 non-null object
Situacao          513 non-null object
dtypes: int64(1), object(6)
memory usage: 32.1+ KB


In [7]:
#Partidos atuais em deputados eleitos

In [35]:
atuais = pd.merge(eleitos_d, deputados_partidos, left_on='Nome_urna', right_on='Nome')

In [36]:
atuais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342 entries, 0 to 341
Data columns (total 12 columns):
Estado            342 non-null object
Cargo             342 non-null object
Nome_candidato    342 non-null object
CPF_x             342 non-null int64
Nome_urna         342 non-null object
Partido           342 non-null object
Situacao          342 non-null object
Link              342 non-null object
CPF_y             342 non-null int64
Nome              342 non-null object
Partido_atual     342 non-null object
Esta_no_cargo     342 non-null object
dtypes: int64(2), object(10)
memory usage: 34.7+ KB


In [37]:
# O cruzamento mostrou 171 nomes faltando - podem ser nomes com nome urna diferente ou suplentes

In [38]:
# Verifica nomes não encontrados

In [39]:
# outer: usa união de chaves de ambos os quadros, semelhante a uma junção externa completa do SQL; classificar chaves lexicograficamente
# indicator: Se True, adiciona uma coluna para saída DataFrame chamado "_merge" com informações sobre a origem de cada linha. Se uma seqüência de caracteres, a coluna com informações sobre a fonte de cada linha será adicionada para saída DataFrame, e a coluna será denominada valor da string. A coluna de informação é de tipo Categorical e assume um valor de "left_only" para observações cuja chave de mesclagem só aparece no DataFrame 'esquerdo', "right_only" para observações cuja chave de mesclagem aparece apenas no DataFrame 'direito' e "ambos" se o A chave de mesclagem da observação é encontrada em ambos.

In [40]:
naoencontrados = pd.merge(eleitos_d, 
                        deputados_partidos, 
                        left_on='Nome_urna', 
                        right_on='Nome',
                        how='outer',
                        indicator=True)

In [41]:
# Agrupa apenas pelo lado direito - a lista do site

In [42]:
rdf = naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [43]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 513 to 683
Data columns (total 12 columns):
Estado            0 non-null object
Cargo             0 non-null object
Nome_candidato    0 non-null object
CPF_x             0 non-null float64
Nome_urna         0 non-null object
Partido           0 non-null object
Situacao          0 non-null object
Link              171 non-null object
CPF_y             171 non-null float64
Nome              0 non-null object
Partido_atual     171 non-null object
Esta_no_cargo     171 non-null object
dtypes: float64(2), object(10)
memory usage: 17.4+ KB


In [44]:
rdf.to_csv('nao_encontrados_partidos_atuais.csv')

In [45]:
# Nesse ponto fiz uma checagem manual dos nomes, verificando quem não era deputado eleito e atualizando os nomes urna

In [46]:
# Repeti o cruzamento e já inclui o CPF para essa base ser cruzada facilmente com outras

In [3]:
xls = pd.ExcelFile('politicos_23_11.xls')

In [4]:
partidos_finais = pd.read_excel(xls, 'politicos_23_11')

In [5]:
partidos_finais.head().reset_index()

,index,Politico_CPF,Politico_Nome_urna,Partido_atual,Esta_no_cargo
0,0,30790590263,MAJOR ROCHA,PSDB,Sim
1,1,50872060772,CÉSAR MESSIAS,PSB,Sim
2,2,13365517391,SIBÁ MACHADO,PT,Não
3,3,44726570234,ALAN RICK,DEM,Sim
4,4,65595564253,JÉSSICA SALES,PMDB,Sim
